# SpaceX Falcon 9 First Stage Landing Prediction: Web Scraping

This notebook performs web scraping to gather SpaceX Falcon 9 and Falcon Heavy launch data from a Wikipedia page. It extracts information from HTML tables and organizes it into a pandas DataFrame.

Key components:

*   BeautifulSoup for HTML parsing
*   Custom helper functions for data extraction
*   Error handling for inconsistent HTML structures
*   Data export to CSV file

## 1. Setup

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
from IPython.display import display

# Constants
WIKI_URL = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

## 2. Helper Functions

In [2]:
def date_time(table_cells):
    """
    Extract date and time from table cells.
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    """
    Extract booster version from table cells.
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out


def landing_status(table_cells):
    """
    Extract landing status from table cells.
    """
    return [i for i in table_cells.strings][0]


def get_mass(table_cells):
    """
    Extract payload mass from table cells.
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass


def extract_column_from_header(row):
    """
    Extract column name from table header.
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name = ' '.join(row.contents)

    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

## 3. Web Scraping Tasks

In [3]:
# Task 1: Request Falcon9 Launch Wiki page
response = requests.get(WIKI_URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Verify page title
print(soup.title)

# Task 2: Extract column names from HTML table header
html_tables = soup.find_all('table')
first_launch_table = html_tables[2]

column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("Extracted column names:", column_names)

# Task 3: Create DataFrame by parsing launch HTML tables
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']

for key in launch_dict.keys():
    launch_dict[key] = []

# Additional columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Parse launch tables
for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')
        if flag:
            launch_dict['Flight No.'].append(flight_number)

            date_time_info = date_time(row[0])
            launch_dict['Date'].append(date_time_info[0].strip(','))
            launch_dict['Time'].append(date_time_info[1])

            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else 'N/A'
            launch_dict['Version Booster'].append(bv)

            launch_dict['Launch site'].append(row[2].a.string if row[2].a else 'N/A')
            launch_dict['Payload'].append(row[3].a.string if row[3].a else 'N/A')
            launch_dict['Payload mass'].append(get_mass(row[4]))
            launch_dict['Orbit'].append(row[5].a.string if row[5].a else 'N/A')
            launch_dict['Customer'].append(row[6].a.string if row[6].a else 'N/A')
            launch_dict['Launch outcome'].append(list(row[7].strings)[0] if row[7].strings else 'N/A')
            launch_dict['Booster landing'].append(landing_status(row[8]) if len(row) > 8 else 'N/A')

# Create DataFrame
df = pd.DataFrame(launch_dict)

# Verify number of rows in DataFrame
num_rows = len(df)
print(f"Number of launches extracted: {num_rows}")
assert num_rows > 100, f"Expected more than 100 launches, but only found {num_rows}"

# Export to CSV
df.to_csv('spacex_web_scraped.csv', index=False)
print("Data exported to 'spacex_web_scraped.csv'")

# Display first few rows and data info
display(df.head())
display(df.info())

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
Extracted column names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Number of launches extracted: 121
Data exported to 'spacex_web_scraped.csv'


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Flight No.       121 non-null    object
 1   Launch site      121 non-null    object
 2   Payload          121 non-null    object
 3   Payload mass     121 non-null    object
 4   Orbit            121 non-null    object
 5   Customer         120 non-null    object
 6   Launch outcome   121 non-null    object
 7   Version Booster  121 non-null    object
 8   Booster landing  121 non-null    object
 9   Date             121 non-null    object
 10  Time             121 non-null    object
dtypes: object(11)
memory usage: 10.5+ KB


None